In [13]:
import pandas as pd
from pymongo import MongoClient

pd.set_option('display.max_colwidth', -1)
pd.options.display.float_format = '${:,.2f}'.format

def _connect_mongo(uri, db):
    """ A util for making a connection to mongo """

    if uri:
#         mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(uri)
    else:
        conn = MongoClient(host, port)
    return conn[db]

def read_mongo(uri, db, collection, query={}, no_id=True, limit=None):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    con = _connect_mongo(uri, db)

    # Make a query to the specific DB and Collection
    ### SET THE LIMIT TO 5 FOR DEV AND TESTING ###
    if limit:
        cursor = con[collection].find(query)
        cursor = cursor[limit]
    else:
        cursor = con[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df = pd.DataFrame(list(cursor))
    # Delete the _id
#     if no_id:
#         del df['_id']
    if limit:
        print(f'number of indexes created has been limited to {limit} ..........................')

    return df

def read_mongo_a(uri, db, collection, limit=None):
    ''' Retrieve data from the Mongo database and transform it to a pandas
    DataFrame; return the DataFrame.

    :param collection: the collection name
    :type collection: string
    :param limit: optional limiter to the number of documents retrieved. '''
    
    database = _connect_mongo(config.uri, db)
    col = {}
    
    # Shorten the cursor length if limit is given, otherwise get everything;
    # transform the retrieved data to a pandas.DataFrame and return it.
    if limit:
        cursor = database[collection].find({})
        cursor = cursor
        for doc in cursor[:limit]:
            col[doc['_id']] = doc
        # Transform the data collection to a dataframe.
        df = pd.DataFrame.from_dict(col, orient='index')
        print(f'The length of your df has been limited to {limit}.')
        return df
    else:
        cursor = database[collection].find({})
        for doc in cursor:
            col[doc['_id']] = doc
        # Transform the data collection to a dataframe.
        df = pd.DataFrame.from_dict(col, orient='index')
        return df
    
    # Let the user know that even though there were no errors the DataFrame
    # was not created.
    print('From read_mongo(): There were no errors, but your dataframe was not \
    created.')

def errors(casts, obs):
    ''' Make a dict of errors for the forecasts. Any dicts in the arguments
    will be flattened before comparison.
    
    :param casts: a list of dictionaries
    :param obs: a dictionary
    
    * For best results all dicts should have all the same keys and subkeys.
    '''
    
    # Flatten all dicts and compare. Add the comparisons to a list and return.
    casts = [overalls.flatten_dict(cast) for cast in casts]
    obs = overalls.flatten_dict(obs)
    return [overalls.compare_dicts(cast, obs) for cast in casts]

def gen_errs_df(df):
    ''' Create an errors dataframe from the argument.
    
    :param df: Must be a pandas DataFrame.
    '''
    ### is there a way to step through three lists together? ###
    errs_list = []
    errs_dict = {}
    # Create the error dicts list to be added to the errs_dict.
    for (obs, casts) in zip(df['weather'], df['forecasts']): ### this creates a the list of errors from each instant
        errs_list.append(errors(casts, obs))
#     df['errs'] = errs
#     print(df.head())
    for (_id, errs) in zip(df['_id'], errs_list):  ### this creates a dict from the errors list with the index as key
        errs_dict[_id] = errs
#     errs_dict
    dd = pd.DataFrame.from_dict(errs_dict, orient='index') ### this puts that dict into DataFrame form
#     print(dd.head())
    for c in dd.columns:  ### this replaces the errors DataFrame dictionaries with a list of their values 
        dd[c] = [list(d.values()) for d in dd[c]]
    return dd


/Users/mastacow/.local/share/virtualenvs/forecast-forecast-GZGA6O4g/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [14]:
import config
import overalls
import weather
import benedict

In [15]:
import sys
print(sys.path)

['/Users/mastacow/data/forecast-forecast/Learn', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python37.zip', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload', '', '/Users/mastacow/.local/share/virtualenvs/forecast-forecast-GZGA6O4g/lib/python3.7/site-packages', '/Users/mastacow/.local/share/virtualenvs/forecast-forecast-GZGA6O4g/lib/python3.7/site-packages/IPython/extensions', '/Users/mastacow/.ipython']


In [16]:
collection = 'legit_inst'
db = 'owmap'
# df = read_mongo(config.uri, db, collection, no_id=False, limit=5)
df = read_mongo_a(config.uri, db, collection, limit=10)

The length of your df has been limited to 10.


In [17]:
# df.set_index(['_id'])
df.drop(labels='_id', axis=1, inplace=True)
df['_id'] = [df['zipcode'][row] + str(df['instant'][row]) for row in range(len(df['instant']))]
# df.set_index('_id', drop=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop(columns=['instant', 'zipcode', '_id'], inplace=True)
df.head()

forecasts  \
0  [{'sunset_time': 0, 'sunrise_time': 0, 'clouds': 15, 'rain': {'3h': 4.94}, 'snow': {}, 'wind': {'speed': 3.2, 'deg': 54}, 'humidity': 89, 'pressure': {'press': 1018, 'sea_level': 1018}, 'temperature': {'temp': 293.04, 'temp_kf': 0, 'temp_max': 293.04, 'temp_min': 293.04}, 'status': 'Rain', 'detailed_status': 'moderate rain', 'weather_code': 501, 'weather_icon_name': '10d', 'visibility_distance': None, 'dewpoint': None, 'humidex': None, 'heat_index': None, 'time_to_instant': 431035}, {'sunset_time': 0, 'sunrise_time': 0, 'clouds': 100, 'rain': {'3h': 1.69}, 'snow': {}, 'wind': {'speed': 3.61, 'deg': 38}, 'humidity': 95, 'pressure': {'press': 1021, 'sea_level': 1021}, 'temperature': {'temp': 288.48, 'temp_kf': 0, 'temp_max': 288.48, 'temp_min': 288.48}, 'status': 'Rain', 'detailed_status': 'light rain', 'weather_code': 500, 'weather_icon_name': '10d', 'visibility_distance': None, 'dewpoint': None, 'humidex': None, 'heat_index': None, 'time_to_instant': 395036}, {'sunset_time': 0, 'sunrise_time': 0, 'clouds': 100, 'rain': {}, 'snow': {}, 'wind': {'speed': 4.6, 'deg': 31}, 'humidity': 86, 'pressure': {'press': 1022, 'sea_level': 1022}, 'temperature': {'temp': 288.11, 'temp_kf': 0, 'temp_max': 288.11, 'temp_min': 288.11}, 'status': 'Clouds', 'detailed_status': 'overcast clouds', 'weather_code': 804, 'weather_icon_name': '04d', 'visibility_distance': None, 'dewpoint': None, 'humidex': None, 'heat_index': None, 'time_to_instant': 380634}, {'sunset_time': 0, 'sunrise_time': 0, 'clouds': 100, 'rain': {}, 'snow': {}, 'wind': {'speed': 4.6, 'deg': 31}, 'humidity': 86, 'pressure': {'press': 1022, 'sea_level': 1022}, 'temperature': {'temp': 288.11, 'temp_kf': 0, 'temp_max': 288.11, 'temp_min': 288.11}, 'status': 'Clouds', 'detailed_status': 'overcast clouds', 'weather_code': 804, 'weather_icon_name': '04d', 'visibility_distance': None, 'dewpoint': None, 'humidex': None, 'heat_index': None, 'time_to_instant': 377038}, {'sunset_time': 0, 'sunrise_time': 0, 'clouds': 100, 'rain': {}, 'snow': {}, 'wind': {'speed': 4.6, 'deg': 31}, 'humidity': 86, 'pressure': {'press': 1022, 'sea_level': 1022}, 'temperature': {'temp': 288.11, 'temp_kf': 0, 'temp_max': 288.11, 'temp_min': 288.11}, 'status': 'Clouds', 'detailed_status': 'overcast clouds', 'weather_code': 804, 'weather_icon_name': '04d', 'visibility_distance': None, 'dewpoint': None, 'humidex': None, 'heat_index': None, 'time_to_instant': 369837}, {'sunset_time': 0, 'sunrise_time': 0, 'clouds': 100, 'rain': {}, 'snow': {}, 'wind': {'speed': 4.6, 'deg': 31}, 'humidity': 86, 'pressure': {'press': 1022, 'sea_level': 1022}, 'temperature': {'temp': 288.11, 'temp_kf': 0, 'temp_max': 288.11, 'temp_min': 288.11}, 'status': 'Clouds', 'detailed_status': 'overcast clouds', 'weather_code': 804, 'weather_icon_name': '04d', 'visibility_distance': None, 'dewpoint': None, 'humidex': None, 'heat_index': None, 'time_to_instant': 366238}, {'sunset_time': 0, 'sunrise_time': 0, 'clouds': 100, 'rain': {}, 'snow': {}, 'wind': {'speed': 4.6, 'deg': 31}, 'humidity': 86, 'pressure': {'press': 1022, 'sea_level': 1022}, 'temperature': {'temp': 288.11, 'temp_kf': 0, 'temp_max': 288.11, 'temp_min': 288.11}, 'status': 'Clouds', 'detailed_status': 'overcast clouds', 'weather_code': 804, 'weather_icon_name': '04d', 'visibility_distance': None, 'dewpoint': None, 'humidex': None, 'heat_index': None, 'time_to_instant': 362637}, {'sunset_time': 0, 'sunrise_time': 0, 'clouds': 100, 'rain': {'3h': 0.55}, 'snow': {}, 'wind': {'speed': 3.84, 'deg': 42}, 'humidity': 94, 'pressure': {'press': 1023, 'sea_level': 1023}, 'temperature': {'temp': 287.46, 'temp_kf': 0, 'temp_max': 287.46, 'temp_min': 287.46}, 'status': 'Rain', 'detailed_status': 'light rain', 'weather_code': 500, 'weather_icon_name': '10d', 'visibility_distance': None, 'dewpoint': None, 'humidex': None, 'heat_index': None, 'time_to_instant': 359038}, {'sunset_time': 0, 'sunrise_time': 0, 'clouds': 100, 'rain': {'3h': 0.55}, 'snow': {}, 'wind': {'speed': 3.84, 'de

In [19]:
df['forecasts'][0]

[{'sunset_time': 0,
  'sunrise_time': 0,
  'clouds': 15,
  'rain': {'3h': 4.94},
  'snow': {},
  'wind': {'speed': 3.2, 'deg': 54},
  'humidity': 89,
  'pressure': {'press': 1018, 'sea_level': 1018},
  'temperature': {'temp': 293.04,
   'temp_kf': 0,
   'temp_max': 293.04,
   'temp_min': 293.04},
  'status': 'Rain',
  'detailed_status': 'moderate rain',
  'weather_code': 501,
  'weather_icon_name': '10d',
  'visibility_distance': None,
  'dewpoint': None,
  'humidex': None,
  'heat_index': None,
  'time_to_instant': 431035},
 {'sunset_time': 0,
  'sunrise_time': 0,
  'clouds': 100,
  'rain': {'3h': 1.69},
  'snow': {},
  'wind': {'speed': 3.61, 'deg': 38},
  'humidity': 95,
  'pressure': {'press': 1021, 'sea_level': 1021},
  'temperature': {'temp': 288.48,
   'temp_kf': 0,
   'temp_max': 288.48,
   'temp_min': 288.48},
  'status': 'Rain',
  'detailed_status': 'light rain',
  'weather_code': 500,
  'weather_icon_name': '10d',
  'visibility_distance': None,
  'dewpoint': None,
  'humidex

In [6]:
for w in df['weather']:
    w.pop('sunset_time', 'sunrise_time')
    w.pop('sunrise_time')
for l in df['forecasts']:
    for f in l:
        f.pop('sunset_time', 'sunrise_time')
        f.pop('sunrise_time')


KeyError: 0

In [17]:
# err_dict = compare_dicts()
errs = []
for forecast, w in zip(df['forecasts'], df['weather']):
#     for cast in forecast:
     errs.append(errors(forecast, w))
df['errs'] = errs
# df.head()

In [18]:
def strip_keys(e):
    ''' Take a list of dictionaries and return a  list of lists of those
    dictionary values. 
    
    :param e: a list of dicts
    :type e: At least in forecast-forecast the input type will be a pandas
    Series, but in general it can take any list-like object of dictionaries.
    :return: a list of lists of dict values w/o keys
    '''
    
    errs = []
    for d in e:
        errs.append([list(overalls.flatten_dict(dic).values()) for dic in d])
    return errs

err_vals = strip_keys(df['errs'])
cast_vals = strip_keys(df['forecasts'])
dd = pd.DataFrame([err_vals, cast_vals], index=['errors', 'forecasts'])
dd = dd.transpose()


In [19]:
dd.head()

,errors,forecasts
0,"[[-1, 0.62, 116, 7, 1, -2.57, -4.55, -0.66, 0,...","[[0, 2.79, 182, 84, 1019, 1019, 289.71, 0, 289..."
1,"[[-1, 0.62, 116, 7, 1, -2.3, -4.55, -0.66, 0, ...","[[0, 2.79, 182, 84, 1019, 1019, 289.71, 0, 289..."
2,"[[-1, 0.62, 116, 7, 1, -2.3, -4.55, -0.66, 0, ...","[[0, 2.79, 182, 84, 1019, 1019, 289.71, 0, 289..."
3,"[[-1, 0.62, 116, 7, 1, -2.3, -4.55, -0.66, 0, ...","[[0, 2.79, 182, 84, 1019, 1019, 289.71, 0, 289..."
4,"[[-1, -0.14, 130, 6, 1, -2.49, -4.45, -1.12, 0...","[[0, 2.45, 184, 83, 1019, 1019, 289.81, 0, 289..."


In [21]:
filename = '~/data/forecast-forecast/Learn/error_set3.csv'
dd.to_csv(filename, float_format='%.3f')

In [8]:
obs = df['weather'][4]
# print(df['weather'][4])
# overalls.flatten(obs)
errs = overalls.compare_dicts(obs, df['weather'][3])
# print(overalls.flatten_dict(errs), '\n')
W = weather.Weather('27606', 'observation', data=errs)
V = weather.Weather('27006', 'observation', data=errs)

{} {}
{} {}
{'speed': 2.59, 'deg': 54} {'speed': 2.17, 'deg': 66}
{'press': 1018, 'sea_level': None} {'press': 1018, 'sea_level': None}
{'temp': 292.3, 'temp_kf': None, 'temp_max': 294.26, 'temp_min': 290.93} {'temp': 292.01, 'temp_kf': None, 'temp_max': 294.26, 'temp_min': 290.37}


In [9]:
overalls.all_values(W.weather)

['DEFAULT',
 0,
 0.42,
 -12,
 0,
 0,
 0.29,
 0.0,
 0.56,
 0,
 0,
 0,
 0,
 'DEFAULT',
 'DEFAULT',
 'DEFAULT',
 44,
 17,
 -2,
 0]

In [10]:
W.weather

{'_id': 'DEFAULT',
 'clouds': 0,
 'rain': {},
 'snow': {},
 'wind': {'speed': 0.42, 'deg': -12},
 'humidity': 0,
 'pressure': {'press': 0},
 'temperature': {'temp': 0.29, 'temp_max': 0.0, 'temp_min': 0.56},
 'status': 0,
 'detailed_status': 0,
 'weather_code': 0,
 'visibility_distance': 0,
 'dewpoint': 'DEFAULT',
 'humidex': 'DEFAULT',
 'heat_index': 'DEFAULT',
 'time_to_instant': 44,
 'sunset_time': 17,
 'sunrise_time': -2,
 'weather_icon_name': 0}

In [11]:
errs = []
for (obs, casts) in zip(df['weather'], df['forecasts']):
    errs.append(errors(casts, obs))


5

In [13]:
# errs[0]

In [15]:
df['errors'] = errs
df.head()

,_id,instant,zipcode,forecasts,weather,errors
5eba42b80923c58fcff13738,5eba42b80923c58fcff13738,1589695200,27152,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674843, 'sunrise_time': 15...","[{'sunset_time': -1589674843, 'sunrise_time': ..."
5eba42b80923c58fcff13739,5eba42b80923c58fcff13739,1589695200,27155,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674843, 'sunrise_time': 15...","[{'sunset_time': -1589674843, 'sunrise_time': ..."
5eba42b80923c58fcff1373a,5eba42b80923c58fcff1373a,1589695200,27157,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674843, 'sunrise_time': 15...","[{'sunset_time': -1589674843, 'sunrise_time': ..."
5eba42b80923c58fcff1373b,5eba42b80923c58fcff1373b,1589695200,27198,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674843, 'sunrise_time': 15...","[{'sunset_time': -1589674843, 'sunrise_time': ..."
5eba42b80923c58fcff1373c,5eba42b80923c58fcff1373c,1589695200,27199,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674860, 'sunrise_time': 15...","[{'sunset_time': -1589674860, 'sunrise_time': ..."
